In [2]:
import numpy as np
import filecmp
import pynq.lib.dma
from pynq import Xlnk
import time
from pynq import Overlay
from pynq import MMIO
ol= Overlay("kociemba_v1.bit")

maxDepthBaseAddress = 0x080
unsolvableBaseAddress = 0x088
coloursBaseAddress = 0x040
orderBaseAddress = 0x090
moveArrayBaseAddress = 0x0a0
mmio=MMIO(0x43C00000,0x10000)

mmio.write(maxDepthBaseAddress, 24)
#print(mmio.read(maxDepthBaseAddress))

def readfile_short(fileName):
    f = open(fileName,'rb')
    buffer= f.read()
    lSize=f.tell()
    buffer_short = np.fromstring(buffer,dtype=np.short)
    open(cmpFile, 'w').close()
    f_out = open(cmpFile,'w+b')
    f_out.write(buffer_short)
    f.close()
    f_out.close()
    return buffer_short, int(lSize/2)

def readfile_char(fileName):
    f = open(fileName,'rb')
    buffer= f.read()
    lSize=f.tell()
    buffer_char = np.fromstring(buffer,dtype=np.byte)
    open(cmpFile, 'w').close()
    f_out = open(cmpFile,'w+b')
    f_out.write(buffer_char)
    f.close()
    f_out.close()
    return buffer_char, lSize
#######################################################################################################################
xlnk = Xlnk()
xlnk.xlnk_reset()

cmpFile = "results/cmp"

list_file_short = ["results/twistMove",\
                   "results/flipMove",\
                   "results/FRtoBR_Move",\
                   "results/URFtoDLF_Move",\
                   "results/URtoDF_Move",\
                   "results/URtoUL_Move",\
                   "results/UBtoDF_Move",\
                   "results/MergeURtoULandUBtoDF"]

list_file_char = ["results/Slice_URFtoDLF_Parity_Prun",\
                  "results/Slice_URtoDF_Parity_Prun",\
                  "results/Slice_Twist_Prun",\
                  "results/Slice_Flip_Prun"]

pruneTableMem=[]

###############################################################################################################################
j =0;
for file in list_file_short:
    pruneTable, size = readfile_short(file)
    if (filecmp.cmp(file,cmpFile)==False):
        print("file compared fail")
    pruneTableMem.append(xlnk.cma_array(shape=(size,), dtype=np.short))
    for i in np.arange(size):
        pruneTableMem[j][i]=pruneTable[i]
    j = j + 1
        
for file in list_file_char:
    pruneTable, size = readfile_char(file)
    if (filecmp.cmp(file,cmpFile)==False):
        print("file compared fail")
    pruneTableMem.append(xlnk.cma_array(shape=(size,), dtype=np.byte))
    for i in np.arange(size):
        pruneTableMem[j][i]=pruneTable[i]
    j = j + 1
    
##############################################################################################################################
pruneTable_address=[0x010,0x018,0x020,0x028,0x030,0x038,0x098,0x0c0,0x0c8,0x0d0,0x0d8,0x0e0 ]

for i in np.arange(len(pruneTable_address)):
    mmio.write(pruneTable_address[i],pruneTableMem[i].physical_address)
    

In [14]:
moves= []

def assign(mNum):
    if(mNum==0):
        moves.append("U")
    elif(mNum==1):
        moves.append("R")
    elif(mNum==2):
        moves.append("F")
    elif(mNum==3):
        moves.append("D")
    elif(mNum==4):
        moves.append("L")
    elif(mNum==5):
        moves.append("B")
    elif(mNum==6):
        moves.append("U'")
    elif(mNum==7):
        moves.append("R'")
    elif(mNum==8):
        moves.append("F'")
    elif(mNum==9):
        moves.append("D'")
    elif(mNum==10):
        moves.append("L'")
    elif(mNum==11):
        moves.append("B'")
    elif(mNum==12):
        moves.append("U2")
    elif(mNum==13):
        moves.append("R2")
    elif(mNum==14):
        moves.append("F2")
    elif(mNum==15):
        moves.append("D2")
    elif(mNum==16):
        moves.append("L2")
    elif(mNum==17):
        moves.append("B2")
    else:
        print("error in coverting value in memory to move")

def bytes(num):
    assign(num& 0xff) 
    assign(num >>8 & 0xff)
    assign(num >> 16 & 0xff)
    assign(num >> 24& 0xff)

def solve(colours):
    for i in np.arange(0,52,4):
        coloursMem = ord(colours[i]) + (ord(colours[i+1])<< 8) + (ord(colours[i+2])<< 16) +(ord(colours[i+3])<< 24)
        mmio.write(coloursBaseAddress+ i ,coloursMem)
    coloursMem= ord(colours[52]) +(ord(colours[53])<< 8)
    mmio.write(coloursBaseAddress+ 52 ,coloursMem)

    mmio.write(0x000, 1)
    start_time = time.time()
    while ((mmio.read(0x000)& 0x2)!=2):
        pass
    elapsed_time = time.time() - start_time
    order= int(mmio.read(orderBaseAddress))
    j=0
    moves.clear()
    if (mmio.read(unsolvableBaseAddress)==1):
        print("Unsolvable")
        return
    while( j<order):
        data = mmio.read(int(moveArrayBaseAddress)+j)
        bytes(data)
        j=j+4
    print("Number of moves is ", order)
    print("elapsed time is ", elapsed_time)
    print(moves[0:order])

In [15]:
solve("DRLUUBFBRBLURRLRUBLRDDFDLFUFUFFDBRDUBRUFLLFDDBFLUBLRBD")
solve("FLBUULFFLFDURRDBUBUUDDFFBRDDBLRDRFLLRLRULFUDRRBDBBBUFL")
solve("BBURUDBFUFFFRRFUUFLULUFUDLRRDBBDBDBLUDDFLLRRBRLLLBRDDF")
solve("UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB")
solve("BBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLL")
solve("LLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDD")
solve("DDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFF")
solve("FFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRR")
solve("RRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUU")

Number of moves is  19
elapsed time is  0.02454209327697754
['D2', "R'", "D'", 'F2', 'B', 'D', 'R2', 'D2', "R'", 'F2', "D'", 'F2', "U'", 'B2', 'L2', 'U2', 'D', 'R2', 'U']
Number of moves is  20
elapsed time is  0.2173149585723877
['U2', 'D', "B'", 'U2', "R'", 'D2', 'L2', 'D', 'R2', 'L', "B'", "U'", 'L2', 'D', 'L2', 'D', 'F2', 'L2', 'B2', "U'"]
Number of moves is  21
elapsed time is  1.6392004489898682
['B', "U'", "L'", "D'", "R'", "D'", 'L2', "D'", 'L', "F'", "L'", 'D', 'F2', 'R2', 'U', 'R2', 'B2', 'U2', 'L2', 'F2', "D'"]
Number of moves is  0
elapsed time is  3.743171691894531e-05
[]
Number of moves is  0
elapsed time is  3.528594970703125e-05
[]
Number of moves is  0
elapsed time is  3.4332275390625e-05
[]
Number of moves is  0
elapsed time is  3.3855438232421875e-05
[]
Number of moves is  0
elapsed time is  3.552436828613281e-05
[]
Number of moves is  0
elapsed time is  3.62396240234375e-05
[]


In [ ]:
#xlnk.xlnk_reset()